### Домашнее задание № 8

**1. Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.**

In [1]:
import numpy as np
import pandas as pd
import catboost as catb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler

***Загрузим датасет из sklearn***

In [2]:
X, y = load_iris(return_X_y=True)

# Провверим размер датасета:
X.shape

(150, 4)

***Отмасштабируем выборку при помощи функции StandardScaler***

In [3]:
X = X.astype(float)
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
X[:10]

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ],
       [-1.50652052,  0.09821729, -1.2833891 , -1.3154443 ],
       [-1.02184904,  1.24920112, -1.34022653, -1.3154443 ],
       [-0.53717756,  1.93979142, -1.16971425, -1.05217993],
       [-1.50652052,  0.78880759, -1.34022653, -1.18381211],
       [-1.02184904,  0.78880759, -1.2833891 , -1.3154443 ],
       [-1.74885626, -0.36217625, -1.34022653, -1.3154443 ],
       [-1.14301691,  0.09821729, -1.2833891 , -1.44707648]])

In [5]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X.T @ X

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения и собственные векторы в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения и собственные векторы в порядке убывания:
(437.77467247979865, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.1045707202106, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.013531335697223, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.1072254642929384, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


Оценим долю дисперсии, которая описывается найденными компонентами.

In [6]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329986, 22.850761786701792, 3.6689218892828745, 0.5178709107154903]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


Таким образом, первая главная компонента описывает почти 73% информации, а первые две в сумме - 95.8%. В то же время последняя компонента описывает всего 0.5% и может быть отброжена без страха значительных потерь в качестве нашего анализа. Мы отбросим последние две компоненты, оставив первые две.

In [7]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [8]:
# Сформируем новую матрицу "объекты-признаки"
Z = X.dot(W)

In [9]:
Z[:10]

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356]])

***Мы получили два датасета - X и Z. Датасет Х имеет по четыре признака для каждого наблбюдения, а датасет Z - по два признака для каждого наблюдения. Потеря информации составляет 4.2%. Обцучим модель CatBoostClassifier на обоих датасетах и сравним метрики.***

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.5, random_state=21)

In [11]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))

In [12]:
model = catb.CatBoostClassifier(n_estimators=300, max_depth=5, 
                                      auto_class_weights='Balanced', 
                                      silent=True, random_state=21, learning_rate=0.1,
                                      early_stopping_rounds=20)
model.fit(X_train, y_train, eval_set=(X_test, y_test),
    verbose=False,
    plot=True)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
y_test_pred_probs = model.predict_proba(X_test)

X_features = get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

TRAIN

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        30

    accuracy                           1.00        75
   macro avg       1.00      1.00      1.00        75
weighted avg       1.00      1.00      1.00        75

TEST

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        30
           1       0.88      0.92      0.90        25
           2       0.89      0.85      0.87        20

    accuracy                           0.93        75
   macro avg       0.93      0.92      0.92        75
weighted avg       0.93      0.93      0.93        75



In [13]:
model.tree_count_

97

In [14]:
X_train, X_test, y_train, y_test = train_test_split(Z, y, shuffle=True, test_size=0.5, random_state=21)

In [15]:
model = catb.CatBoostClassifier(n_estimators=300, max_depth=5, 
                                      auto_class_weights='Balanced', 
                                      silent=True, random_state=21, learning_rate=0.1,
                                      early_stopping_rounds=20)
model.fit(X_train, y_train, eval_set=(X_test, y_test),
    verbose=False,
    plot=True)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
y_test_pred_probs = model.predict_proba(X_test)

Z_features = get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

TRAIN

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        30

    accuracy                           1.00        75
   macro avg       1.00      1.00      1.00        75
weighted avg       1.00      1.00      1.00        75

TEST

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        30
           1       0.80      0.96      0.87        25
           2       0.94      0.75      0.83        20

    accuracy                           0.91        75
   macro avg       0.91      0.89      0.90        75
weighted avg       0.92      0.91      0.91        75



In [16]:
model.tree_count_

81

#### ВЫВОД: потеря 4.2% информации не оказала значительного влияния на работу модели и на качество предсказания. Небольшая разница в  accuracy может быть связана с небольшим количеством наблюдений в датасете и с неравномерным распределением классов (у нас три класса) в выборках. Стоит обратить внимание на то, что время работы модели на датасете с двумя признаками в 2.4 раза меньше, чем на датасете с четырьмя признаками - 36ms против 86ms. Таким образом, разумное уменьшение размерности данных приводит к существенному ускорению работы модели и при аккуратном подходе к отбору признаков не сильно влияет на качество предсказания. 